## 0) Loading

In [1]:
import os

os.environ['COMET_API_KEY'] = "rGzWoLsJvI7QFeL6tHfQjwCPj"

import comet_ml

# Commet Init
comet_ml.init(project_name="DNA_Deberta_Small", api_key= "rGzWoLsJvI7QFeL6tHfQjwCPj")

COMET INFO: Comet API key is valid
COMET INFO: Comet API key saved in /home/jovyan/.comet.config


In [2]:
from datasets import load_dataset
datasets = load_dataset("simecek/Human_DNA_v0_DNABert6tokenized")
datasets.set_format("torch")
datasets

Using custom data configuration simecek--Human_DNA_v0_DNABert6tokenized-9a684042f2db6cd1
Reusing dataset parquet (/home/jovyan/.cache/huggingface/datasets/simecek___parquet/simecek--Human_DNA_v0_DNABert6tokenized-9a684042f2db6cd1/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 95417
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 858737
    })
})

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification, DataCollatorForLanguageModeling

tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")

In [4]:
tokenizer.decode([0,1,2,3,4,5])

'[PAD] [UNK] [CLS] [SEP] [MASK] AAAAAA'

## 1) Training

In [5]:
from transformers import DebertaConfig, DebertaForMaskedLM, TrainingArguments, Trainer

model_config = DebertaConfig(vocab_size=len(tokenizer.vocab), max_position_embeddings=512, num_hidden_layers=6)
model_config

DebertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": null,
  "position_biased_input": true,
  "relative_attention": false,
  "transformers_version": "4.19.2",
  "type_vocab_size": 0,
  "vocab_size": 4101
}

In [6]:
model = DebertaForMaskedLM(config=model_config)
sum(p.numel() for p in model.parameters()) // 10**6

46

In [7]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

In [8]:
# training_args = TrainingArguments(
#     output_dir='./model',          # output directory to where save model checkpoint
#     evaluation_strategy="steps",    # evaluate each `logging_steps` steps
#     overwrite_output_dir=True,      
#     num_train_epochs=10,            # number of training epochs, feel free to tweak
#     per_device_train_batch_size=10, # the training batch size, put it as high as your GPU memory fits
#     gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
#     per_device_eval_batch_size=64,  # evaluation batch size
#     logging_steps=1000,             # evaluate, log and save model checkpoints every 1000 step
#     save_steps=1000,
#     # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
#     # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
# )



training_args = TrainingArguments(
    output_dir='./model',          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=2,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=64, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=1,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps=2500,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=5000,
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=5,           # whether you don't have much space so you let only 5 model weights saved in the disk
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=datasets['train'],
    eval_dataset=datasets['test'],
)

In [10]:
trainer.train()

/home/jovyan/my-conda-envs/myEnv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 858737
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 26836
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/home/jovyan' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.ml htt

Step,Training Loss,Validation Loss
2500,7.365400,7.285398
5000,7.272600,7.274663
7500,7.261100,7.266738
10000,7.254000,7.259422
12500,7.248900,7.255401
15000,7.245600,7.249977
17500,7.241000,7.245810
20000,7.238200,7.245580
22500,7.234600,7.241339
25000,7.232500,7.233794


***** Running Evaluation *****
  Num examples = 95417
  Batch size = 64
***** Running Evaluation *****
  Num examples = 95417
  Batch size = 64
Saving model checkpoint to ./model/checkpoint-5000
Configuration saved in ./model/checkpoint-5000/config.json
Model weights saved in ./model/checkpoint-5000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 95417
  Batch size = 64
***** Running Evaluation *****
  Num examples = 95417
  Batch size = 64
Saving model checkpoint to ./model/checkpoint-10000
Configuration saved in ./model/checkpoint-10000/config.json
Model weights saved in ./model/checkpoint-10000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 95417
  Batch size = 64
***** Running Evaluation *****
  Num examples = 95417
  Batch size = 64
Saving model checkpoint to ./model/checkpoint-15000
Configuration saved in ./model/checkpoint-15000/config.json
Model weights saved in ./model/checkpoint-15000/pytorch_model.bin
***** Running Evaluation *****
  Num 

TrainOutput(global_step=26836, training_loss=7.256896053808317, metrics={'train_runtime': 24876.7783, 'train_samples_per_second': 69.039, 'train_steps_per_second': 1.079, 'total_flos': 2.275066275145298e+17, 'train_loss': 7.256896053808317, 'epoch': 2.0})

In [11]:
import torch 
#torch.cuda.empty_cache() 

## 3) Evaluation and export

In [18]:
from huggingface_hub import notebook_login

notebook_login()

In [20]:
model.push_to_hub("DNADeberta")

/home/jovyan/my-conda-envs/myEnv/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:39: FutureWarning: Pass token='DNADeberta' as keyword args. From version 0.8 passing these as positional arguments will result in an error
  warnings.warn(
/home/jovyan/my-conda-envs/myEnv/lib/python3.8/site-packages/huggingface_hub/hf_api.py:673: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(


OSError: Looks like you do not have git-lfs installed, please install. You can install from https://git-lfs.github.com/. Then run `git lfs install` (you only have to do this once).

In [ ]:
# tokenizer.push_to_hub("cDNABERT_v0")

In [13]:
trainer.push_to_hub

<bound method Trainer.push_to_hub of <transformers.trainer.Trainer object at 0x7fa044c01700>>